# Data ingestion

[ES API](https://zincsearch-docs.zinc.dev/api-es-compatible/search/search/)

In [1]:
import os
import yaml
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

conf_dir = os.environ['CONFIG_DIR']
data_dir = os.environ['DATA_DIR']

load_dotenv(os.path.join(conf_dir, '.env'))

config_path = os.path.join(conf_dir,'data_config.yml')
config = {}
with open(config_path, "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
index_config = config['elastic_index_settings']
print(index_config)
            
index_name = 'index99'

{'mappings': {'properties': {'doc_id': {'type': 'text'}, 'content': {'type': 'text'}, 'category': {'type': 'text'}, 'content_len': {'type': 'integer'}}}}


In [6]:
config['elastic_index_settings'] = {
        "name": index_name,
        "storage_type": "disk",
        "shard_num": 1,
        "mappings": {
            "properties": {
                "doc_id": {
                    "type": "text",
                    "index": True,
                    "store": True,
                    "highlightable": True
                },
                "content": {
                    "type": "text",
                    "index": True,
                    "store": True,
                    "highlightable": True
                },
                "category": {
                    "type": "keyword",
                    "index": True,
                    "sortable": True,
                    "aggregatable": True
                },
                "content_len": {
                    "type": "integer",
                    "index": True,
                    "sortable": True,
                    "aggregatable": False
                }
            }
        },
        "settings": {
            "analysis": {
                "analyzer": {
                    "default": {
                        "type": "standard"
                    }
                }
            }
        }
    }

In [7]:
# with open(config_path, "w") as f:
#     yaml.dump(config, f, default_flow_style=False)

In [2]:
from light_retrieval_flow.data_ingestion import create_index

create_index(index_name, index_config)

Index 'index99' created successfully.


In [3]:
from light_retrieval_flow.data_ingestion import read_csv_as_dicts, load_bulk_documents

csv_data_path = os.path.join(data_dir, 'pipelines_data', 'knowledgebase.csv')
index_entries = read_csv_as_dicts(csv_data_path)
load_bulk_documents(index_name, index_entries)

Num entries: 9654
Document loaded successfully.


# Data retrieval

In [5]:
from light_retrieval_flow.data_ingestion import search_documents, pretty

print(index_name)
pretty(search_documents(index_name, 'xiaomi', category='news'))

index99


[{'content': 'After Microsoft signed a deal to test Windows 10 on Xiaomi devices in 2015 and then Xiaomi bought a trove of patents to help run other Microsoft services on its devices in 2016, today the two companies announced another chapter in its collaboration. Xiaomi and Microsoft have signed a Strategic Framework Memorandum of Understanding (MoU) to work more closely in the areas of cloud computing, AI (including Microsoft’s Cortana business) and hardware. To date, Xiaomi has largely focused its mobile phone and wider device strategy in Asia Pacific, where Gartner revealed yesterday that it (and Huawei) were the only two vendors to increase their market shares at a time of general decline. This deal could point to how Xiaomi is looking to raise its game in the West, specifically in the US, especially as it eyes up an IPO that has been reported to potentially value the company at $100 billion. On the side of Microsoft, it’s particularly interesting given that the company has largely